<a href="https://colab.research.google.com/github/Heba2424/PDF-Question-Answering-System/blob/main/RAG_based_PDF_Question_Answering_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain faiss-cpu gradio --quiet
!pip install langchain_openai langchain_community --quiet
!pip install -U langchain-openai --quiet
!pip install openai --quiet
!pip install langchain sentence-transformers faiss-cpu --quiet
!!pip install PyPDF2 langchain sentence-transformers faiss-cpu --quiet
# Install necessary packages
!pip install gradio PyPDF2 langchain sentence-transformers faiss-cpu --quiet
!pip install transformers --quiet
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 400.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.

In [2]:
import requests
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.vectorstores import FAISS
from transformers import pipeline

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Step 1: Download the PDF
pdf_url = "https://static.latexstudio.net/article/2018/0912/neuralnetworksanddeeplearning.pdf"
response = requests.get(pdf_url)

# Save the PDF locally
pdf_path = "neural_networks_and_deep_learning.pdf"
with open(pdf_path, "wb") as file:
    file.write(response.content)


In [4]:
# Step 2: Read the PDF
pdf_text = ""
with open(pdf_path, "rb") as file:
    reader = PyPDF2.PdfReader(file)
    num_pages = len(reader.pages)
    for page_num in range(num_pages):
        page = reader.pages[page_num]
        pdf_text += page.extract_text()

if not pdf_text.strip():
    raise ValueError("No text extracted from the PDF. Please check the PDF content.")

In [5]:
# Step 3: Preprocess and Split the Text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(pdf_text)
documents = [{"page_content": chunk} for chunk in chunks]

if len(documents) == 0:
    raise ValueError("No chunks were created. Please check the text splitting process.")
else:
    print(f"Number of chunks created: {len(documents)}")

Number of chunks created: 720


In [6]:
# Step 4: Create embeddings using SentenceTransformer model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(model_name)

# Function to embed text
def embed_text(text):
    return embedding_model.encode(text, convert_to_numpy=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Step 5: Build the FAISS vector store
try:
    chunk_embeddings = np.array([embed_text(chunk["page_content"]) for chunk in documents])
    dimension = chunk_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # Create a FAISS index
    index.add(chunk_embeddings)
    document_store = {i: documents[i] for i in range(len(documents))}
    print("FAISS vector store created successfully.")
except Exception as e:
    raise RuntimeError(f"Error creating FAISS vector store: {e}")

FAISS vector store created successfully.


In [8]:
# Step 6: Initialize a Generative Model (RAG)
# Using HuggingFace's GPT-2 pipeline as an example
generator = pipeline('text-generation', model='gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [9]:
# Step 7: Tokenizer and Model setup for generating responses
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Adjust to the model you're using
generator = AutoModelForCausalLM.from_pretrained("gpt2")  # Adjust model


In [10]:
# Function to generate text from retrieved chunks
def generate_response(retrieved_chunks, query, max_input_length=512):
    # Combine retrieved chunks into a context
    context = " ".join([chunk["page_content"] for chunk in retrieved_chunks])
    input_text = f"Context: {context}\nQuestion: {query}\nAnswer: "

    # Tokenize input text and truncate if necessary
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=max_input_length)

    # Generate output, specifying max_new_tokens to control how many tokens are generated
    generated_output = generator.generate(**inputs, max_new_tokens=100)

    # Decode the generated tokens back to text
    generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)

    return generated_text


In [11]:
# Step 8: Query the vector store and Generate Response
query = "What are the main concepts covered in the book?"
query_embedding = embed_text(query)

try:
    # Perform similarity search
    distances, indices = index.search(np.array([query_embedding]), k=5)

    # Retrieve top chunks
    retrieved_chunks = [document_store[idx] for idx in indices[0]]

    # Generate a response using the retrieved chunks and the query
    response = generate_response(retrieved_chunks, query)

    print("Generated Response:")
    print(response)
except NameError:
    print("The vector store was not created due to an earlier error.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Response:
Context: ideas such as backpropagation, regularization, the softmax function, and so on. However, to
read the chapter you don’t need to have worked in detail through all the earlier chapters. It
will, however, help to have read Chapter 1, on the basics of neural networks. When I use
concepts from Chapters 2 to 5, I provide links so you can familiarize yourself, if necessary.
It’s worth noting what the chapter is not. It’s not a tutorial on the latest and greatest
neural networks libraries. Nor are we going to be training deep networks with dozens of
layers to solve problems at the very leading edge. Rather, the focus is on understanding
some of the core principles behind deep neural networks, and applying them in the simple,
easy-to-understand context of the MNIST problem. Put another way: the chapter is not
going to bring you right up to the frontier. Rather, the intent of this and earlier chapters is to recognition problems. And you will have a foundation to use n

In [13]:
import gradio as gr

# Step 8: Query the vector store and Generate Response
def gradio_interface(query):
    # Assuming `index`, `document_store`, `embedding_model`, `tokenizer`, and `generator` are already initialized
    query_embedding = embed_text(query)

    try:
        # Perform similarity search in FAISS index
        distances, indices = index.search(np.array([query_embedding]), k=5)

        # Retrieve top chunks from the document store
        retrieved_chunks = [document_store[idx] for idx in indices[0]]

        # Generate a response using the retrieved chunks and the query
        response = generate_response(retrieved_chunks, query)

        return response
    except Exception as e:
        return f"An error occurred: {e}"

# Create the Gradio interface
interface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Enter your query"),
    outputs="text",
    title="PDF Query Answering",
    description="Ask a question based on the PDF data processed and stored in FAISS. The system will retrieve relevant chunks and generate an answer."
)

# Launch the Gradio app
interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://92c11890e9068a1347.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
